# CloudEx Algo Main Script

#### Default Imports

In [14]:
import sys 
sys.path

new_paths = ['/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/root/CloudExchange', '/root/']
for p in new_paths : 
    sys.path.append(p) 


In [15]:
sys.path

['/root/cs349f/implementation1',
 '/root/CloudExchange/bazel-bin/python/',
 '/root/.pyenv/versions/3.7.7/lib/python37.zip',
 '/root/.pyenv/versions/3.7.7/lib/python3.7',
 '/root/.pyenv/versions/3.7.7/lib/python3.7/lib-dynload',
 '',
 '/home/steam1994/.local/lib/python3.7/site-packages',
 '/root/.pyenv/versions/3.7.7/lib/python3.7/site-packages',
 '/home/steam1994/.local/lib/python3.7/site-packages/IPython/extensions',
 '/home/steam1994/.ipython',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/root/CloudExchange',
 '/root/',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/root/CloudExchange',
 '/root/']

In [16]:
# Import packages.
import datetime
import json
import os
import sys
import time

import pandas as pd
import numpy as np
import redis
from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# CloudEx imports.
import cloud_ex

# Import AlgorithmicTrader helper class.
from algorithmic_trader_shay import AlgorithmicTrader
from algorithmic_trader_shay import summarize_historical_trades_df

# Start Redis and its Python API.
os.system("redis-server --daemonize yes")
time.sleep(1)

# Get CloudEX and VM-specific config. 
# NOTE: gateway_ip will be null when the exchange is not online 
def get_vm_config():
    with open("/root/vm_config.json", "r") as read_file:
        config = json.load(read_file)
    return config

config = get_vm_config()


# utilities  

ORDER_FIELDS_LIST = [
    'Symbol', 'OrderID', 'CancelID', 'ClientID', 'OrderType', 'OrderAction',
    'SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
    'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice', 'ResultType','NumShares'
]

TRADE_FIELDS_LIST = [
    "Symbol", "BuyerSerialNum", "SellerSerialNum", "BuyerOrderID",
    "SellerOrderID", "BuyerClientID", "SellerClientID", "ExecPrice",
    "CashTraded", "SharesTraded", "CreationTimestamp", "ReleaseTimestamp",
    "TradeSerialNum"
]

'''
Takes in a cloud_ex.VectorOrder with serialized orders and returns a DataFrame
'''
def OrderDF(order_vec):
    if not len(order_vec):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = pd.DataFrame(order_vec).applymap(lambda x:x.SerializeOrder())[0].str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.VectorOrder with serialized trades and returns a DataFrame
'''
def TradeDF(trade_vec):
    if not len(trade_vec):
        return pd.DataFrame(columns=TRADE_FIELDS_LIST)
    df = pd.DataFrame(trade_vec).applymap(lambda x:x.SerializeTrade())[0].str.split('|', expand=True)
    df.columns = TRADE_FIELDS_LIST
    for label in ["ExecPrice", "CashTraded", "SharesTraded",
                  "CreationTimestamp", "ReleaseTimestamp", "TradeSerialNum"]:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.MapStringOrder mapping Order ID strings to outstanding the coorresponding orders, 
and returns a DataFrame
'''
def OutstandingOrderDF(outstanding_orders):
    if not len(outstanding_orders):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = (pd.DataFrame(outstanding_orders.items())[1]).apply(lambda x:x.SerializeOrder()).str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df 

In [17]:
config

{'gateway_ip': '10.128.15.220',
 'client_id': 'C15',
 'client_token': 'dyjpqwzyvminsvjpxtyasxdnzpnqkuux',
 'project_id': 'jinkun-pro1',
 'bigtable_id': 'test-bt',
 'table_name': 'record-tb'}

#### Custom Imports

In [18]:
import importlib 
import sys 
#importlib.reload(sys.modules['mean_reversion_shay'])
def reload(r) : 
    importlib.reload(sys.modules[r])

In [19]:
import strategies_shay   
import threading
import utilities as u 
default_symbols = ['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DJ', 'DK', 'DL', 'DM', 'DN', 'DO', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DU', 'DV']
default_top_symbols = ['DV', 'CQ', 'DN', 'CL', 'CT', 'BF', 'CW', 'AW', 'DS', 'CS']

In [20]:
trader = None

#### Get the trader object

In [21]:
def getTrader() : 
    global trader
    if trader : 
        u.debug("Returning saved trader")
        return trader 

    # Get relevant fields from VM-specific config. Token is yours only, so don't make it public.
    gateway_ip = config["gateway_ip"]
    client_id = config["client_id"]
    client_token = config["client_token"]

    # Clear any existing data locally.
    redis_api = redis.Redis()
    redis_api.flushall();

    # Create CloudEx base trader object.
    trader = cloud_ex.Trader(gateway_ip, client_id, client_token)
    return trader 

def getSymbols() : 
    trader = getTrader() 
    return trader.GetSymbols()


def getPortfolio(): 
    portfolio_mat = cloud_ex.MapStringInt()
    return trader.GetPortfolioMatrix(portfolio_mat)
    

In [14]:
T = getTrader()

Returning saved trader


In [15]:
config['gateway_ip']

'10.128.15.220'

#### Main Todos



#### TRADING PARAMETERS

In [24]:
GLOBALS = { 
    'NUM_SHARES' :  1 , 
    'BIN_INTERVAL_MS' : 1000,  #interval to bin the data with 
    'WAIT_INTERVAL_SECONDS' : 1, 
    'BACKTEST_LOOKBACK_PERIOD_SECONDS' : 3*60  , #amount of historical data to backtest on 
    'MAX_NUM_ORDERS' : 60 , #how long the algo will trade for, in # of bins 
} 

### Selecting symbols to trade on based on aggregate volume over last n seconds  

We need to figure out which symbols to trade on prior to the initiation of trading. It seems natural to rank them by some metric and then take the top N of them, for example volume 

In [23]:
import numpy as np

def aggregate_volume(symbol, seconds_in_past) : 
    """
    Gets the most recent 'seconds_in_past' historical data for the symbol and compute the total 
    'CashTraded' 
    """
    u.debug("Getting aggregate volume for {}".format(symbol))
    end_time_ms = int(time.time()*1e3)
    start_time_ms = end_time_ms - int(seconds_in_past*1e3)
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
    sym_df = TradeDF(symbol_trades_vec)
    #print(sym_df)
    print("Returning volume for symbol:" + symbol)
    return np.sum(sym_df['CashTraded'])    

def periodicity_metric(symbol, seconds_in_past, normalize_with_dc = True): 
    """
    Find symbols with high degrees of periodicity.
    """
    u.debug("Getting periodicity of {}".format(symbol))
    end_time_ms = int(time.time()*1e3)
    start_time_ms = end_time_ms - int(seconds_in_past*1e3)
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
    sym_df = TradeDF(symbol_trades_vec)
    sym_df.to_csv("periodicity_" + str(symbol) + "_" + str(time.time()) +".csv", sep=',')
    close_price_vector = sym_df['ExecPrice']

    ps = np.abs(np.fft.fft(close_price_vector))**2
    freqs = np.fft.fftfreq(len(close_price_vector), GLOBALS['BIN_INTERVAL_MS']/10**3)
    if normalize_with_dc:
        periodicity = np.max(ps[1:])/float(np.sum(ps))
    else:
        periodicity = np.max(ps[1:])/float(np.sum(ps[1:]))

    print(symbol, periodicity)
    return periodicity, freqs

def multithreaded_volume_request(syms,seconds_in_past) : 
    import concurrent.futures
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [] 
        for sym in syms : 
            print("Submitting thread for sym: " + sym )
            future = executor.submit(aggregate_volume, sym, seconds_in_past)
            futures.append(future)
        
    # now we get the results 
    values = [future.result() for future in futures ] 
    return values 
    
def get_volume_for_symbol_in_thread(symbol, seconds_in_past) : 
    # create the thread 
    t = threading.Thread(target=aggregate_volume,
                         args=(symbol,seconds_in_past))
    # start the thread 
    t.start() 
    # return it 
    return t 


def ranked_volume_symbols(symbols , seconds_in_past) : 
    
    ## -- 
    threads = [ get_volume_for_symbol_in_thread(symbol,seconds_in_past) for symbol in symbols  ] 
    
    for thread in threads : 
        thread.join() 
    
    
    ## -- 

def rank_symbols_by_volume(symbols , seconds_in_past) : 
    data = [ [symbol, aggregate_volume(symbol,seconds_in_past)  ] for  symbol in symbols ] 
    data.sort(key=lambda x:  x[1] , reverse=True )
    return data

def get_top_n_symbols_by_volume(symbols,seconds_in_past, n )  : 
    ranked = rank_symbols_by_volume(symbols,seconds_in_past) 
    syms = [ x[0] for x in ranked[0:n]] 
    return (syms , ranked)  

def rank_symbols_by_periodicity(symbols , seconds_in_past) : 
    data = [ [symbol, periodicity_metric(symbol, seconds_in_past)  ] for  symbol in symbols ] 
    data.sort(key=lambda x:  x[1][0], reverse=True )
    return data

def get_top_n_symbols_by_periodicity(symbols,seconds_in_past, n )  : 
    ranked = rank_symbols_by_periodicity(symbols, seconds_in_past) 
    syms = [ x[0] for x in ranked[0:n]] 
    return (syms , ranked)

In [27]:
syms = ['AE', 'AN','AB', 'AI' ,'AO'] 
seconds = 60*60

ts1= time.time()
multithreaded_volume_request(syms, seconds)
te1= time.time() 

ts2= time.time() 
for sym in syms : 
    aggregate_volume(sym,seconds)
te2= time.time() 

print("Multithreaded took {} seconds".format(te1-ts1))
print("Singlethreaded took {} seconds".format(te2-ts2))


Submitting thread for sym: AE
Getting aggregate volume for AE
Submitting thread for sym: AN
Getting aggregate volume for AN
Submitting thread for sym: AB
Getting aggregate volume for AB
Submitting thread for sym: AI
Getting aggregate volume for AI
Submitting thread for sym: AO
Getting aggregate volume for AO
Returning volume for symbol:AEReturning volume for symbol:AN

Returning volume for symbol:AI
Returning volume for symbol:AB
Returning volume for symbol:AO
Getting aggregate volume for AE
Returning volume for symbol:AE
Getting aggregate volume for AN
Returning volume for symbol:AN
Getting aggregate volume for AB
Returning volume for symbol:AB
Getting aggregate volume for AI
Returning volume for symbol:AI
Getting aggregate volume for AO
Returning volume for symbol:AO
Multithreaded took 81.81353855133057 seconds
Singlethreaded took 78.71345901489258 seconds


#### Conclusion from above is that multithreaded market data request is not helpful 

In [33]:
top_symbols, symbol_ranks = get_top_n_symbols_by_volume(default_symbols, 60*3, 10)

Getting aggregate volume for AA
Getting aggregate volume for AB
Getting aggregate volume for AC
Getting aggregate volume for AD
Getting aggregate volume for AE
Getting aggregate volume for AF
Getting aggregate volume for AG
Getting aggregate volume for AH
Getting aggregate volume for AI
Getting aggregate volume for AJ
Getting aggregate volume for AK
Getting aggregate volume for AL
Getting aggregate volume for AM
Getting aggregate volume for AN
Getting aggregate volume for AO
Getting aggregate volume for AP
Getting aggregate volume for AQ
Getting aggregate volume for AR
Getting aggregate volume for AS
Getting aggregate volume for AT
Getting aggregate volume for AU
Getting aggregate volume for AV
Getting aggregate volume for AW
Getting aggregate volume for AX
Getting aggregate volume for AY
Getting aggregate volume for AZ
Getting aggregate volume for BA
Getting aggregate volume for BB
Getting aggregate volume for BC
Getting aggregate volume for BD
Getting aggregate volume for BE
Getting 

In [19]:
most_periodic, periodicity_ranks = get_top_n_symbols_by_periodicity(default_symbols, 5*60, 10)

Getting periodicity of AA
AA 3.941409499092487e-05
Getting periodicity of AB
AB 0.0021347010313830864
Getting periodicity of AC
AC 0.0010364418804461838
Getting periodicity of AD
AD 6.139716957544517e-05
Getting periodicity of AE
AE 2.4159801096135368e-05
Getting periodicity of AF
AF 0.0001935302274454138
Getting periodicity of AG
AG 0.0005176673134732748
Getting periodicity of AH
AH 0.00012944375742601362
Getting periodicity of AI
AI 6.295000173807019e-05
Getting periodicity of AJ
AJ 8.197315683868433e-05
Getting periodicity of AK
AK 0.000613077686391594
Getting periodicity of AL
AL 0.0006672104819759397
Getting periodicity of AM
AM 0.00010047192135413004
Getting periodicity of AN
AN 0.00012260678703078737
Getting periodicity of AO
AO 2.4712433414776992e-05
Getting periodicity of AP
AP 7.768926983654538e-05
Getting periodicity of AQ
AQ 0.0006161710888670642
Getting periodicity of AR
AR 0.00011044236345124613
Getting periodicity of AS
AS 0.00016317373100600203
Getting periodicity of AT

In [20]:
most_periodic

['AB', 'DC', 'AU', 'AC', 'BH', 'BS', 'AL', 'AQ', 'AK', 'BL']

In [21]:
periodicity_ranks

[['AB',
  (0.0021347010313830864,
   array([ 0.        ,  0.0025974 ,  0.00519481,  0.00779221,  0.01038961,
           0.01298701,  0.01558442,  0.01818182,  0.02077922,  0.02337662,
           0.02597403,  0.02857143,  0.03116883,  0.03376623,  0.03636364,
           0.03896104,  0.04155844,  0.04415584,  0.04675325,  0.04935065,
           0.05194805,  0.05454545,  0.05714286,  0.05974026,  0.06233766,
           0.06493506,  0.06753247,  0.07012987,  0.07272727,  0.07532468,
           0.07792208,  0.08051948,  0.08311688,  0.08571429,  0.08831169,
           0.09090909,  0.09350649,  0.0961039 ,  0.0987013 ,  0.1012987 ,
           0.1038961 ,  0.10649351,  0.10909091,  0.11168831,  0.11428571,
           0.11688312,  0.11948052,  0.12207792,  0.12467532,  0.12727273,
           0.12987013,  0.13246753,  0.13506494,  0.13766234,  0.14025974,
           0.14285714,  0.14545455,  0.14805195,  0.15064935,  0.15324675,
           0.15584416,  0.15844156,  0.16103896,  0.16363636,  0.1

In [14]:
getSymbols()

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. cloud_ex.Trader(arg0: str, arg1: str, arg2: str)

Invoked with: None, 'C15', 'dyjpqwzyvminsvjpxtyasxdnzpnqkuux'

In [41]:
top_symbols

['CX', 'CZ', 'AH', 'DA', 'AA', 'AD', 'CS', 'AG', 'AO', 'AP']

In [36]:
symbol_ranks

[['CX', 1810092],
 ['CZ', 1755790],
 ['AH', 1250054],
 ['DA', 1223190],
 ['AA', 952611],
 ['AD', 833049],
 ['CS', 791593],
 ['AG', 612493],
 ['AO', 491351],
 ['AP', 398533],
 ['AL', 397636],
 ['AC', 360296],
 ['AB', 324539],
 ['AE', 257818],
 ['DU', 244409],
 ['DC', 233574],
 ['AQ', 222745],
 ['AM', 137258],
 ['AT', 131034],
 ['BG', 102394],
 ['CT', 69741],
 ['DN', 67146],
 ['AF', 66522],
 ['AS', 65161],
 ['DV', 64797],
 ['AK', 59043],
 ['DO', 58409],
 ['BA', 58294],
 ['CU', 57585],
 ['DR', 57445],
 ['DS', 55721],
 ['BB', 55399],
 ['AW', 54619],
 ['DD', 52422],
 ['AX', 51844],
 ['CW', 51681],
 ['AZ', 50923],
 ['DB', 50808],
 ['BF', 49356],
 ['DT', 49302],
 ['DI', 48363],
 ['DE', 47453],
 ['BE', 46780],
 ['CR', 45690],
 ['DF', 45351],
 ['DH', 45262],
 ['DG', 45033],
 ['CV', 44878],
 ['DK', 44838],
 ['DJ', 42193],
 ['AY', 41792],
 ['BH', 41535],
 ['BK', 40999],
 ['BI', 39919],
 ['CY', 39208],
 ['AV', 38277],
 ['BD', 38029],
 ['BM', 37681],
 ['BC', 36847],
 ['DQ', 36776],
 ['BT', 35077],


### QUERY ORDER HISTORY

In [14]:
### Order Monitoring 
def outstanding_orders() : 
    outstanding_orders = cloud_ex.MapStringOrder()
    trader.GetOutstandingOrders(outstanding_orders)
    u.debug("You have {} outstanding orders.".format(len(outstanding_orders)))
    # Transform outstanding orders into a DataFrame
    outstanding_orders = OutstandingOrderDF(outstanding_orders)
    return outstanding_orders
    
def historical_orders() : 
    my_historical_orders = cloud_ex.VectorOrder()
    trader.GetAllHistoricalOrders(my_historical_orders)
    u.debug("You have submitted a total of {} order(s).".format(len(my_historical_orders))) 
    my_historical_orders_df = OrderDF(my_historical_orders)
    return my_historical_orders_df

def historical_trades() : 
    my_historical_trades = cloud_ex.VectorTrade()
    trader.GetAllHistoricalTrades(my_historical_trades)
    u.debug("You have made a total of {} trade(s).".format(len(my_historical_trades)))
    my_historical_trades_df = TradeDF(my_historical_trades)
    return my_historical_trades_df

In [15]:
outstanding_orders()

You have 0 outstanding orders.


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares


# 1. Backtesting

Let's see how we can backtest our trading algorithms to get them ready for live trading. In the following cells we will download historical data and evaluate how well a mean reversion trader would have done.

## 1.1 Get our bank of strategies

### Note on strategy and strategy parameters terminology

A strategy is implemented with specific parameters. <b>The pair of the strategy and parameters will be called an "algo"</b>
The backtest logic will take a dictionary of string keys (identifiers) to a tuple of (strategy , params). This dictionary will be called "algo_bank" 

For example => 

In [22]:
# NOte a fully specified strategy is called an "algo" and consists of a tuple of (strategy, kwargs) where kwargs provide
# the parameters for the strategy 

to_reload = ["algorithmic_trader_shay", 
             "mean_reversion_shay",
             "momentum_shay", 
             "random_buy_shay", 
             "random_sell_shay", 
             "buy_strategy_shay", 
             "sell_strategy_shay", 
             "strategies_shay" ] 

for r in to_reload : 
    reload(r) 
    
def mean_reversion_algo(ma,t) : 
    # Helper function for making the "algo" data structure
    # NOTE this returns a TUPLE of (strategy , kwargs)
    return (strategies_shay.strategies['mean_reversion'].strategy, { 'ma' : ma, 'threshold' : t} ) 

def momentum_algo(p1,p2,t) : 
    # NOTE this returns a TUPLE of (strategy , kwargs)
    return (strategies_shay.strategies['momentum'].strategy, { 'p1' : p1, 'p2' : p2, 'threshold' : t} ) 

def random_buy_algo(p) : 
    return (strategies_shay.strategies['random_buy'].strategy , { 'p' : p} ) 

def random_sell_algo(p) : 
    return (strategies_shay.strategies['random_sell'].strategy , { 'p' : p} ) 



# Define an "Algo Bank", which holds the (strategy, kwargs) pair, indexed by a unique identifier 
algo_bank = { 
    'mr_10_3' : mean_reversion_algo(10,3),
    'mr_10_5' : mean_reversion_algo(10,5), 
    'mr_10_10' : mean_reversion_algo(10,10),
    'mo_5_5_1' : momentum_algo(0.5,0.5,1) , 
    'mo_5_5_3' : momentum_algo(0.5,0.5,3) , 
    'mo_8_2_1' : momentum_algo(0.8,0.2,1) , 
    'mo_8_2_3' : momentum_algo(0.8,0.2,3) , 
    'rb_10'    : random_buy_algo(0.1) , 
    'BUY'      : random_buy_algo(1.0) , 
    'SELL'     : random_sell_algo(1.0) , 
    'rb_50'    : random_buy_algo(0.5) , 
    'rs_10'    : random_sell_algo(0.1) , 
    'rs_50'    : random_sell_algo(0.5) , 
 
}


def backtest_strategy_with_symbol(data,strategy, symbol, params ) :  
    """
    Backtests a given strategy, kwarg pair on a diven symbol using the last seconds_in_past seconds of historical data 
    """
   
    
    #bin_interval_ms = 500
    #summarize_historical_trades_df(symbol_historical_trades_df, bin_interval_ms)
    
    # Initial capital and shares we will use for the backtest. 
    init_capital = 100000
    init_shares = 1000

    # Set up trading algorithm parameters.
    num_shares = 10

    # Set up the strategy 
    trader = None
    algo = strategy(trader, [symbol],GLOBALS['BIN_INTERVAL_MS'])  # instantiates the strategy which is subclass of algorithmic_trader

    # Run the backtest.
    roi, action_list = algo.backtest(data,
                 num_shares, 
                 init_capital, 
                 init_shares, 
                 **params)

    u.debug("Algo ROI={}%".format(roi))
    return roi,action_list
    
    

def backtest_algobank_on_symbols(algobank, symbols) : 
    """
    Backtests all algos ( strategy, kwarg pairs) on each symbol in symbols, over the last seconds_in_past seconds 
    of data 
    
    Returns the results in a sorted list with algos ranked by roi 
    """
    results = [] 
    
    
    for symbol in symbols : 
        
        u.debug("\n\nBacktesting symbol: {}".format(symbol) ) 
        
        end_time_ms = int(time.time()*1e3)
        start_time_ms = end_time_ms - int(GLOBALS['BACKTEST_LOOKBACK_PERIOD_SECONDS']*1e3)
        symbol_trades_vec = cloud_ex.VectorTrade()  
        cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
        u.debug("There are a total of {} trades for {} symbol".format(len(symbol_trades_vec), symbol))
        symbol_historical_trades_df = TradeDF(symbol_trades_vec)
        symbol_historical_trades_df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
    
        for (algoname,algo) in algobank.items() : 
            strategy, params = algo
            u.debug("algo={}".format(algoname))
            roi,action_list = backtest_strategy_with_symbol(symbol_historical_trades_df,strategy, symbol, params)
            results.append( [  roi, symbol, algoname,action_list ])
            u.debug("Made {} trades with roi: {}".format(len(action_list), roi))
    
    results.sort(key=lambda x : x[0], reverse=True) 
    return results         

            

In [29]:
end_time_ms = int(time.time()*1e3)
start_time_ms = end_time_ms - int(60*60*3*1e3)
for symbol in ['CX', 'CZ', 'AH', 'DA', 'AA', 'AD', 'CS', 'AG', 'AO', 'AP'] : 
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                            config['table_name'], symbol, start_time_ms, 
                                            end_time_ms, symbol_trades_vec)
    u.debug("There are a total of {} trades for {} symbol".format(len(symbol_trades_vec), symbol))
    symbol_historical_trades_df = TradeDF(symbol_trades_vec)
    df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
    df.to_csv("{}_whole_day_2.csv".format(symbol))

There are a total of 8757 trades for CX symbol
There are a total of 9041 trades for CZ symbol
There are a total of 10107 trades for AH symbol
There are a total of 9027 trades for DA symbol
There are a total of 12153 trades for AA symbol
There are a total of 14066 trades for AD symbol
There are a total of 9277 trades for CS symbol
There are a total of 11515 trades for AG symbol
There are a total of 9984 trades for AO symbol
There are a total of 11686 trades for AP symbol


In [13]:
df.to_csv("AE_timeseries_day_2.csv")

In [17]:
backtest_symbols = ['AT' , 'AG' ,'AJ', 'AQ', 'AN']

In [18]:
backtest_results = backtest_algobank_on_symbols(algo_bank, backtest_symbols)



Backtesting symbol: AT
There are a total of 1829 trades for AT symbol
algo=mr_10_3
Running in offline mode. Could not set active symbols.
Algo ROI=26.24793388429751%
Made 122 trades with roi: 26.24793388429751
algo=mr_10_5
Running in offline mode. Could not set active symbols.
Algo ROI=26.909090909090907%
Made 58 trades with roi: 26.909090909090907
algo=mr_10_10
Running in offline mode. Could not set active symbols.
Algo ROI=33.31404958677686%
Made 9 trades with roi: 33.31404958677686
algo=mo_5_5_1
Running in offline mode. Could not set active symbols.
Algo ROI=39.14049586776861%
Made 174 trades with roi: 39.14049586776861
algo=mo_5_5_3
Running in offline mode. Could not set active symbols.
Algo ROI=39.24793388429754%
Made 32 trades with roi: 39.24793388429754
algo=mo_8_2_1
Running in offline mode. Could not set active symbols.
Algo ROI=38.239669421487605%
Made 178 trades with roi: 38.239669421487605
algo=mo_8_2_3
Running in offline mode. Could not set active symbols.
Algo ROI=38.586

In [19]:
backtest_results

[[108.22689075630251,
  'AN',
  'rb_50',
  [(2, 'Buy', 22.0),
   (3, 'Buy', 20.0),
   (4, 'Buy', 22.0),
   (8, 'Buy', 25.0),
   (9, 'Buy', 25.0),
   (10, 'Buy', 26.0),
   (11, 'Buy', 26.0),
   (13, 'Buy', 26.0),
   (14, 'Buy', 27.0),
   (15, 'Buy', 28.0),
   (18, 'Buy', 28.0),
   (20, 'Buy', 27.0),
   (21, 'Buy', 29.0),
   (22, 'Buy', 29.0),
   (24, 'Buy', 29.0),
   (25, 'Buy', 28.0),
   (26, 'Buy', 29.0),
   (27, 'Buy', 29.0),
   (28, 'Buy', 32.0),
   (29, 'Buy', 31.0),
   (30, 'Buy', 31.0),
   (31, 'Buy', 32.0),
   (34, 'Buy', 31.0),
   (35, 'Buy', 29.0),
   (36, 'Buy', 31.0),
   (38, 'Buy', 27.0),
   (40, 'Buy', 31.0),
   (41, 'Buy', 31.0),
   (45, 'Buy', 35.0),
   (47, 'Buy', 36.0),
   (51, 'Buy', 34.0),
   (58, 'Buy', 34.0),
   (60, 'Buy', 33.0),
   (62, 'Buy', 34.0),
   (66, 'Buy', 38.0),
   (68, 'Buy', 42.0),
   (70, 'Buy', 40.0),
   (75, 'Buy', 39.0),
   (76, 'Buy', 39.0),
   (77, 'Buy', 38.0),
   (78, 'Buy', 39.0),
   (81, 'Buy', 41.0),
   (85, 'Buy', 40.0),
   (89, 'Buy', 40.

In [25]:
info = [ (x[0], x[1],x[2]) for x in backtest_results ]

In [26]:
info

[(108.22689075630251, 'AN', 'rb_50'),
 (92.84285714285713, 'AJ', 'rb_50'),
 (71.56302521008405, 'AN', 'mo_5_5_1'),
 (70.17647058823528, 'AN', 'mo_8_2_1'),
 (69.73109243697479, 'AN', 'rb_10'),
 (64.92436974789916, 'AN', 'mo_5_5_3'),
 (63.613445378151255, 'AN', 'mo_8_2_3'),
 (59.96428571428572, 'AJ', 'mo_5_5_1'),
 (59.16428571428571, 'AJ', 'rb_10'),
 (58.447552447552454, 'AQ', 'rb_50'),
 (55.842975206611555, 'AT', 'rb_50'),
 (53.10924369747897, 'AN', 'mr_10_10'),
 (51.878571428571405, 'AJ', 'mo_8_2_1'),
 (51.571428571428584, 'AJ', 'mo_8_2_3'),
 (50.985714285714266, 'AJ', 'mo_5_5_3'),
 (50.27857142857144, 'AJ', 'mr_10_10'),
 (49.78991596638656, 'AN', 'rs_10'),
 (43.62937062937061, 'AQ', 'mo_8_2_1'),
 (43.529411764705884, 'AN', 'mr_10_5'),
 (43.2027972027972, 'AQ', 'mo_5_5_1'),
 (43.042857142857144, 'AJ', 'mr_10_5'),
 (42.1888111888112, 'AQ', 'rb_10'),
 (42.099999999999994, 'AJ', 'rs_10'),
 (40.97520661157026, 'AT', 'rb_10'),
 (39.651515151515156, 'AG', 'rb_50'),
 (39.27272727272728, 'AQ',

# Deploying the trading threads 

Worked more on this and completed it -- see <b>trade_algorithmically</b> function below. Nothing is tested yet so will expect bugs. But for now each thread will log its own submitted order ids to a logfile named by algo+timestamp. 

In [ ]:
test_ranking = [[11, 'CQ', 'mo_8_2_3', []],
     [10,'CQ', 'mo_8_2_1', [] ] , 
     [8, 'CQ', 'mo_5_5_1', [] ] , 
     [6,'CQ', 'mr_10_10', []], 
     [6, 'CQ', 'mr_10_5', []],
     [4, 'CQ', 'mr_10_3', []]] 


In [26]:
def run_and_evaluate_algorithm(**kwargs) : 
    """
    Intended as target of new thread() 
    1. Launches the algo on the symbol and starts trading 
    2. should calculate ROI of the algo 
    4. Writes ROI and submitted order ids to disk 
    """
    name     = kwargs['name']
    strategy = kwargs['strategy']  
    strategy_parameters = kwargs['strategy_parameters'] 
    num_shares = kwargs['num_shares'] 
    max_num_orders = kwargs['max_num_orders'] 
    symbol   = kwargs['symbol']
    trader   = kwargs['trader'] #reference to the 1 trader instance connnected to cloudX
    
    # create the AlgorithmicTrader Object 
    algo = strategy(trader, [symbol], bin_interval_ms=GLOBALS['BIN_INTERVAL_MS']) 
    
    # get and set id for this trader  (for logging purposes)
    trader_id = name + "_" + str(time.time()).split(".")[0]
    #print(trader_id) 
    #print(type(algo))
    #algo.super().set_id(trader_id)  #added this method 
    
    # Calculate portfolio state pre-trading 
    pass 

    # start and finish trading
    order_ids = algo.trade(symbol,num_shares,max_num_orders,GLOBALS['WAIT_INTERVAL_SECONDS'] ,trader_id=trader_id,**strategy_parameters)
    # can SIMULATE if we want 
    
    # calculate portfolio state post-trading
    pass 

    #time.sleep(10)
    # write the order ids to a log file 
    u.logfile(trader_id + "_order_ids", json.dumps(order_ids) )
    # write the ROI information a log file 
    pass     

def run_algorithm_in_thread(kwargs) : 
    # create the thread 
    t = threading.Thread(target=run_and_evaluate_algorithm,
                         kwargs=kwargs)
    # start the thread 
    t.start() 
    # return it 
    return t 

def deploy_top_N_algorithms(trader,ranked_algos, N,  num_shares, max_num_orders) : 
    sublist = ranked_algos[0:N] 
    ts = []
    for to_deploy in sublist : 
        roi, symbol, algoname , _  =  to_deploy 
        strategy, strategy_parameters = algo_bank[algoname]  
        
        arguments = { 
            'name' : algoname, 
            'strategy' : strategy, 
            'strategy_parameters' : strategy_parameters, 
            'num_shares' : num_shares, 
            'max_num_orders' : max_num_orders,  
            'symbol' : symbol , 
            'trader' : trader ,
        }
        ts.append(run_algorithm_in_thread(arguments))
        
    return ts 


def trade_algorithmically(trader,algobank,symbols,N=5) : 
    while True : 
        u.logfile("tradeloop", "\n{}, Doing backtest".format(time.time()))
        ranked = backtest_algobank_on_symbols(algobank,symbols)
        
        fname = "backtest_results_{}".format(time.time())
        u.logfile(fname, json.dumps(ranked))
        u.logfile("tradeloop","backtest_results=>")
        u.logfile("tradeloop", "\n{}\n".format(json.dumps(ranked[0:N])))
        
        u.logfile("tradeloop", "\n{}, Launching Trading".format(time.time()))
        algo_threads = deploy_top_N_algorithms(trader,ranked,N, GLOBALS['NUM_SHARES'] , GLOBALS['MAX_NUM_ORDERS']) 
        for t in algo_threads : 
            t.join() 

In [ ]:
trade_algorithmically(getTrader(), algo_bank,['AN','AG','AP', 'AI', 'AH', 'AC', 'AE', 'AT'],N=5)

Returning saved trader


Backtesting symbol: AN
There are a total of 371 trades for AN symbol
algo=mr_10_3
Running in offline mode. Could not set active symbols.
Algo ROI=0.2087912087911974%
Made 10 trades with roi: 0.2087912087911974
algo=mr_10_5
Running in offline mode. Could not set active symbols.
Algo ROI=0.0549450549450512%
Made 2 trades with roi: 0.0549450549450512
algo=mr_10_10
Running in offline mode. Could not set active symbols.
Algo ROI=0.0%
Made 0 trades with roi: 0.0
algo=mo_5_5_1
Running in offline mode. Could not set active symbols.
Algo ROI=0.0274725274725256%
Made 24 trades with roi: 0.0274725274725256
algo=mo_5_5_3
Running in offline mode. Could not set active symbols.
Algo ROI=0.0659340659340728%
Made 6 trades with roi: 0.0659340659340728
algo=mo_8_2_1
Running in offline mode. Could not set active symbols.
Algo ROI=0.021978021978029005%
Made 22 trades with roi: 0.021978021978029005
algo=mo_8_2_3
Running in offline mode. Could not set active symbols.
Algo ROI=0.06593

Algo ROI=3.1872509960159334%
Made 0 trades with roi: 3.1872509960159334
algo=mo_8_2_1
Running in offline mode. Could not set active symbols.
Algo ROI=3.1872509960159334%
Made 0 trades with roi: 3.1872509960159334
algo=mo_8_2_3
Running in offline mode. Could not set active symbols.
Algo ROI=3.1872509960159334%
Made 0 trades with roi: 3.1872509960159334
algo=rb_10
Running in offline mode. Could not set active symbols.
Algo ROI=3.410358565737056%
Made 16 trades with roi: 3.410358565737056
algo=BUY
Running in offline mode. Could not set active symbols.
Algo ROI=5.107569721115539%
Made 65 trades with roi: 5.107569721115539
algo=SELL
Running in offline mode. Could not set active symbols.
Algo ROI=0.6135458167330796%
Made 100 trades with roi: 0.6135458167330796
algo=rb_50
Running in offline mode. Could not set active symbols.
Algo ROI=4.529880478087648%
Made 65 trades with roi: 4.529880478087648
algo=rs_10
Running in offline mode. Could not set active symbols.
Algo ROI=2.8286852589641427%
Mad

Algo ROI=1.0819672131147513%
Made 19 trades with roi: 1.0819672131147513
algo=rs_50
Running in offline mode. Could not set active symbols.
Algo ROI=1.262295081967224%
Made 82 trades with roi: 1.262295081967224


Backtesting symbol: AG
There are a total of 419 trades for AG symbol
algo=mr_10_3
Running in offline mode. Could not set active symbols.
Algo ROI=1.0515463917525807%
Made 3 trades with roi: 1.0515463917525807
algo=mr_10_5
Running in offline mode. Could not set active symbols.
Algo ROI=1.0309278350515427%
Made 0 trades with roi: 1.0309278350515427
algo=mr_10_10
Running in offline mode. Could not set active symbols.
Algo ROI=1.0309278350515427%
Made 0 trades with roi: 1.0309278350515427
algo=mo_5_5_1
Running in offline mode. Could not set active symbols.
Algo ROI=0.9536082474226788%
Made 13 trades with roi: 0.9536082474226788
algo=mo_5_5_3
Running in offline mode. Could not set active symbols.
Algo ROI=1.0309278350515427%
Made 1 trades with roi: 1.0309278350515427
algo=mo_8_2_1
R

Algo ROI=2.1276595744680833%
Made 0 trades with roi: 2.1276595744680833
algo=mo_5_5_1
Running in offline mode. Could not set active symbols.
Algo ROI=2.074468085106389%
Made 26 trades with roi: 2.074468085106389
algo=mo_5_5_3
Running in offline mode. Could not set active symbols.
Algo ROI=2.1276595744680833%
Made 0 trades with roi: 2.1276595744680833
algo=mo_8_2_1
Running in offline mode. Could not set active symbols.
Algo ROI=2.0531914893616943%
Made 20 trades with roi: 2.0531914893616943
algo=mo_8_2_3
Running in offline mode. Could not set active symbols.
Algo ROI=2.1223404255319167%
Made 1 trades with roi: 2.1223404255319167
algo=rb_10
Running in offline mode. Could not set active symbols.
Algo ROI=2.3617021276595835%
Made 15 trades with roi: 2.3617021276595835
algo=BUY
Running in offline mode. Could not set active symbols.
Algo ROI=4.180851063829792%
Made 112 trades with roi: 4.180851063829792
algo=SELL
Running in offline mode. Could not set active symbols.
Algo ROI=-0.047872340425

Algo ROI=5.8358974358974365%
Made 21 trades with roi: 5.8358974358974365
algo=BUY
Running in offline mode. Could not set active symbols.
Algo ROI=9.64102564102565%
Made 103 trades with roi: 9.64102564102565
algo=SELL
Running in offline mode. Could not set active symbols.
Algo ROI=0.7179487179487296%
Made 100 trades with roi: 0.7179487179487296
algo=rb_50
Running in offline mode. Could not set active symbols.
Algo ROI=8.461538461538453%
Made 90 trades with roi: 8.461538461538453
algo=rs_10
Running in offline mode. Could not set active symbols.
Algo ROI=4.317948717948724%
Made 21 trades with roi: 4.317948717948724
algo=rs_50
Running in offline mode. Could not set active symbols.
Algo ROI=1.7487179487179532%
Made 93 trades with roi: 1.7487179487179532


Backtesting symbol: AP
There are a total of 309 trades for AP symbol
algo=mr_10_3
Running in offline mode. Could not set active symbols.
Algo ROI=0.0%
Made 0 trades with roi: 0.0
algo=mr_10_5
Running in offline mode. Could not set active s

Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=mr_10_5
Running in offline mode. Could not set active symbols.
Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=mr_10_10
Running in offline mode. Could not set active symbols.
Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=mo_5_5_1
Running in offline mode. Could not set active symbols.
Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=mo_5_5_3
Running in offline mode. Could not set active symbols.
Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=mo_8_2_1
Running in offline mode. Could not set active symbols.
Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=mo_8_2_3
Running in offline mode. Could not set active symbols.
Algo ROI=0.8333333333333286%
Made 0 trades with roi: 0.8333333333333286
algo=rb_10
Running in offline mode. Could not set active symbols.
Algo ROI=0.974999

Algo ROI=-5.088669950738904%
Made 11 trades with roi: -5.088669950738904
algo=mo_8_2_3
Running in offline mode. Could not set active symbols.
Algo ROI=-5.418719211822662%
Made 0 trades with roi: -5.418719211822662
algo=rb_10
Running in offline mode. Could not set active symbols.
Algo ROI=-5.970443349753694%
Made 16 trades with roi: -5.970443349753694
algo=BUY
Running in offline mode. Could not set active symbols.
Algo ROI=-9.665024630541879%
Made 99 trades with roi: -9.665024630541879
algo=SELL
Running in offline mode. Could not set active symbols.
Algo ROI=-1.1330049261083701%
Made 100 trades with roi: -1.1330049261083701
algo=rb_50
Running in offline mode. Could not set active symbols.
Algo ROI=-8.305418719211815%
Made 91 trades with roi: -8.305418719211815
algo=rs_10
Running in offline mode. Could not set active symbols.
Algo ROI=-4.862068965517238%
Made 20 trades with roi: -4.862068965517238
algo=rs_50
Running in offline mode. Could not set active symbols.
Algo ROI=-2.5812807881773

In [ ]:
ts = deploy_top_N_algorithms(getTrader(), backtest_results, 2, GLOBALS['NUM_SHARES'], 60)
for t in ts : 
    t.join()

In [ ]:
ts = deploy_top_N_algorithms(None, test_ranking, 5, GLOBALS['NUM_SHARES'], GLOBALS['MAX_NUM_ORDERS'])
for t in ts : 
    t.join()

In [ ]:
import importlib 
importlib.reload(sys.modules['random_buy_shay'])
importlib.reload(sys.modules['algorithmic_trader_shay'])

In [ ]:
historical_orders()